In [1]:
import pandas as pd
import warnings

# Use the context manager to temporarily suppress warnings
# with warnings.catch_warnings():
#     # Filter out the warnings you want to ignore by category
#     warnings.filterwarnings("ignore", category=SomeWarningCategory)

# Dataset Merging

Here we will aggregate various datasets from OECD database.

In [2]:
# read public finance dataset, use this as the base dataframe
base_df = pd.read_csv('data/public_finance.csv')

# Merge Population Data

In [3]:
# read population csv to dataframe
pop_df = pd.read_csv('data/population.csv')

# rename columns
pop_df.rename(columns={'LOCATION': 'iso3', 'Time': 'year', 'Value': 'total_population'}, inplace=True)

In [4]:
# merge
base_df = base_df.merge(pop_df[['iso3', 'year', 'total_population']], on=['iso3', 'year'], how='left')

## Pivot and Merge Nonhealth Predictors

In [5]:
#read dataframe
nonhealth_df = pd.read_csv('data/nonhealth_predictors.csv')

# Pivot method
pivot_nonhealth_df = nonhealth_df.pivot(index=['COU', 'Year'], columns='Variable', values='Value').reset_index()

# If you want to have a flat DataFrame without a MultiIndex in the columns
pivot_nonhealth_df.columns.name = None  # Remove the categories name
pivot_nonhealth_df = pivot_nonhealth_df.reset_index()  # To flatten the DataFrame

In [6]:
# rename columns
# Create a dictionary of old and new column names
rename_dict = {
    'COU': 'iso3',
    'Year': 'year',
    'Alcohol consumption': 'alcohol_consume',
    'Fruits supply': 'fruit_supply',
    'Obese population, measured': 'obese_pop_measured',
    'Overweight population, measured': 'overweight_pop_measured',
    'Sugar supply': 'sugar_supply',
    'Tobacco consumption': 'tobacco_consumption',
    'Total calories supply': 'total_calories_supply',
    'Total fat supply': 'total_fat_supply',
    'Total protein supply': 'total_protein_supply',
    'Vaping population: e-cigarettes use, adults (aged 15+)': 'vape_pop_15_over',
    'Vegetables supply': 'veggie_supply'
}

# rename
pivot_nonhealth_df.rename(columns=rename_dict, inplace=True)

In [7]:
# drop index column
pivot_nonhealth_df.drop(pivot_nonhealth_df.columns[[0]], axis=1, inplace=True)

In [8]:
# Merge pivoted nonhealth df to base_df
base_df = base_df.merge(pivot_nonhealth_df, on=['iso3', 'year'], how='left')

## Merge Life Expectancy Data

In [9]:
# read life expectancy csv to dataframe
lifeexp_df = pd.read_csv('data/life_expectancy.csv')

# rename columns
lifeexp_df.rename(columns={'COU': 'iso3', 'Year': 'year', 'Value': 'life_expectancy'}, inplace=True)

In [10]:
# merge
base_df = base_df.merge(lifeexp_df[['iso3', 'year', 'life_expectancy']], on=['iso3', 'year'], how='left')

## Merge Health Expenditures Data

In [11]:
# read health expenditure csv to dataframe
healthexp_df = pd.read_csv('data/health_expenditures.csv')

# rename columns
healthexp_df.rename(columns={'LOCATION': 'iso3', 'Year': 'year', 'Value': 'health_exp_pct_gdp'}, inplace=True)

In [12]:
# merge
base_df = base_df.merge(healthexp_df[['iso3', 'year', 'health_exp_pct_gdp']], on=['iso3', 'year'], how='left')

## Merge GDP Per Capital Data

In [13]:
# read health expenditure csv to dataframe
gdp_df = pd.read_csv('data/gdp_per_capita.csv')

# rename columns
gdp_df.rename(columns={'COU': 'iso3', 'Year': 'year', 'Value': 'gdp_per_capita'}, inplace=True)

In [14]:
# merge
base_df = base_df.merge(gdp_df[['iso3', 'year', 'gdp_per_capita']], on=['iso3', 'year'], how='left')

### Save Fully Merged Dataframe

In [15]:
base_df.head()

,iso3,year,expitem1,expitem10,expitem11,expitem2,expitem3,expitem4,expitem5,expitem6,...,sugar_supply,tobacco_consumption,total_calories_supply,total_fat_supply,total_protein_supply,vape_pop_15_over,veggie_supply,life_expectancy,health_exp_pct_gdp,gdp_per_capita
0,AUS,1945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AUS,1946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AUS,1947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AUS,1948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AUS,1949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# finally, save end df to new .csv
base_df.to_csv('oecd_data.csv', index=False)

# Data Cleaning

Now we can start cleaning with the fully merged dataframe.

In [17]:
# read fully merged df
df = pd.read_csv('oecd_data.csv')

# Filter Rows

In [18]:
# Filter rows within the range of 1990 to 2023
df = df[(df['year'] >= 1990) & (df['year'] <= 2023)]

## Drop Columns

In [19]:
# List of columns to remove
# columns_to_remove = [
#     'expitem8', 'expitem9', 'expitem10', 'expitem11',
#     'revitem1', 'revitem2', 'revitem3', 'revitem4', 'revitem5',
#     'revitem6', 'revitem7', 'revitem8', 'revitem9', 'revitem10',
#     'exch', 'gap', 'nlgq', 'nlgqa', 'ypg', 'ypga', 'yrg', 'yrga', 'ggflm'
# ]

# Remove the specified columns
# df = df.drop(columns=columns_to_remove)

## Rename Columns

In [20]:
column_name_mapping = {
    'expitem1': 'education_exp',
    'expitem2': 'public_health_exp',
    'expitem3': 'wages_exp',
    'expitem4': 'pensions_exp',
    'expitem5': 'sickness_disability_exp',
    'expitem6': 'unemployment_exp',
    'expitem7': 'family_children_exp',
    'expitem8': 'subsidies_exp',
    'expitem9': 'public_investment_exp',
    'expitem10': 'other_primary_exp',
    'expitem11': 'property_income_exp',
    'expitem11_primary': 'property_income_paid',
    'revitem1': 'personal_income_tax',
    'revitem2': 'social_security_contrib',
    'revitem3': 'corporate_income_tax',
    'revitem4': 'environmental_tax',
    'revitem5': 'other_consumption_tax',
    'revitem6': 'immovable_property_tax',
    'revitem7': 'other_property_tax',
    'revitem8': 'sales_goods_services',
    'revitem9': 'other_nonproperty_tax',
    'revitem10': 'property_income',
    'revitem10_primary': 'property_income_received',
    'gdpv': 'gdp_volume_market_prices',
    'exch': 'exchange_rate_usd',
    'gap': 'total_economy_output_gap',
    'nlgq': 'gov_net_lending_gdp_percentage',
    'nlgqa': 'gov_net_lending_adj_gdp_percentage',
    'ypg': 'current_disbursements_gen_gov',
    'ypga': 'cyclically_adj_current_disbursements_gen_gov',
    'yrg': 'current_receipts_gen_gov',
    'yrga': 'cyclically_adj_current_receipts_gen_gov',
    'ggflm': 'gross_public_debt_maastricht_value'
}

# Rename the columns using the dictionary
df = df.rename(columns=column_name_mapping)

# Map Country Names to ISO3

In [21]:
# assign full country names to country codes
# find unique country codes
unique_iso3_values = df['iso3'].unique()
print(unique_iso3_values)

['AUS' 'AUT' 'BEL' 'BGR' 'CAN' 'CHE' 'CHL' 'COL' 'CRI' 'CZE' 'DEU' 'DNK'
 'ESP' 'EST' 'FIN' 'FRA' 'GBR' 'GRC' 'HUN' 'IRL' 'ISL' 'ISR' 'ITA' 'JPN'
 'KOR' 'LTU' 'LUX' 'LVA' 'MEX' 'NLD' 'NOR' 'NZL' 'POL' 'PRT' 'SVK' 'SVN'
 'SWE' 'TUR' 'USA']


In [22]:
country_code_to_name = {
    'AUS': 'Australia', 'AUT': 'Austria', 'BEL': 'Belgium', 'BGR': 'Bulgaria', 
    'CAN': 'Canada', 'CHE': 'Switzerland', 'CHL': 'Chile', 'COL': 'Colombia', 
    'CRI': 'Costa Rica', 'CZE': 'Czech Republic', 'DEU': 'Germany', 'DNK': 'Denmark', 
    'ESP': 'Spain', 'EST': 'Estonia', 'FIN': 'Finland', 'FRA': 'France', 
    'GBR': 'United Kingdom', 'GRC': 'Greece', 'HUN': 'Hungary', 'IRL': 'Ireland', 
    'ISL': 'Iceland', 'ISR': 'Israel', 'ITA': 'Italy', 'JPN': 'Japan', 
    'KOR': 'South Korea', 'LTU': 'Lithuania', 'LUX': 'Luxembourg', 'LVA': 'Latvia', 
    'MEX': 'Mexico', 'NLD': 'Netherlands', 'NOR': 'Norway', 'NZL': 'New Zealand', 
    'POL': 'Poland', 'PRT': 'Portugal', 'SVK': 'Slovakia', 'SVN': 'Slovenia', 
    'SWE': 'Sweden', 'TUR': 'Turkey', 'USA': 'United States'
}

# map the country name to the corresponding iso3
df['country'] = df['iso3'].map(country_code_to_name)

In [23]:
#### CONTINENT AND COUNTRY
geo_df = pd.read_csv('data/geographical.csv')

# Rename the column in geo_df
geo_df.rename(columns={'Country Name': 'country', 'Continent': 'continent', 'Geographical Region': 'region'}, inplace=True)

# Merge the DataFrames on the 'country' column
df = pd.merge(df, geo_df[['country', 'continent', 'region']], on='country', how='left')

In [24]:
# Define a function to move columns to specific positions
def move_columns(df, columns_to_move, positions):
    for column, position in zip(columns_to_move, positions):
        df.insert(position, column, df.pop(column))

# Define the columns to move and their respective positions
columns_to_move = ["country", "region", "continent", "gdp_per_capita", "health_exp_pct_gdp"]
positions = [df.columns.get_loc("iso3"), df.columns.get_loc("iso3") + 1, df.columns.get_loc("iso3") + 2, df.columns.get_loc("gdp") + 1, df.columns.get_loc("year") + 1]

# Move the columns using the function
move_columns(df, columns_to_move, positions)

# Data Transformation

In [25]:
# Define a list of expense columns to calculate percentages for
expenditure_columns = [
    'education_exp', 'public_health_exp', 'wages_exp', 'pensions_exp',
    'sickness_disability_exp', 'unemployment_exp', 'family_children_exp',
    'subsidies_exp', 'public_investment_exp', 'other_primary_exp', 'property_income_exp'
]

# Define a list of revenue columns to calculate percentages for
revenue_columns = [
    'personal_income_tax', 'social_security_contrib', 'corporate_income_tax',
    'environmental_tax', 'other_consumption_tax', 'immovable_property_tax',
    'other_property_tax', 'sales_goods_services', 'other_nonproperty_tax',
    'property_income'
]

rev_exp_columns = expenditure_columns + revenue_columns

In [26]:
# Calculate percentages and create new variables with "_pct_gdp" suffix
for col in rev_exp_columns:
    new_col_name = col + '_pct_gdp'
    
    # Check if both 'col' and 'gdp' are not missing (not NaN)
    valid_indices = df[col].notna() & df['gdp'].notna()
    # Perform the calculation only for valid indices
    df.loc[valid_indices, new_col_name] = (df.loc[valid_indices, col] / df.loc[valid_indices, 'gdp'] * 100).round(2)

# Check if all columns in expenditure_columns and revenue are not missing for each row
valid_expenditure_indices = df[expenditure_columns].notna().all(axis=1)
valid_revenue_indices = df[revenue_columns].notna().all(axis=1)

# Calculate the sum of expenditure and revenue for rows where all expenditure columns are not missing
df.loc[valid_expenditure_indices, 'total_expenditure'] = df.loc[valid_expenditure_indices, expenditure_columns].sum(axis=1)
df.loc[valid_revenue_indices, 'total_revenue'] = df.loc[valid_revenue_indices, revenue_columns].sum(axis=1)


# Calculate the expense and revenue as a percentage of GDP
# Create a boolean mask to check if both 'total' and 'gdp' are not missing for each row
valid_exp_indices = df['total_expenditure'].notna() & df['gdp'].notna()
valid_rev_indices = df['total_revenue'].notna() & df['gdp'].notna()

# Calculate 'total_expense_pct_gdp' and 'total_revenue_pct_gdp' only for rows where both 'total' and 'gdp' are not missing
df.loc[valid_exp_indices, 'total_expense_pct_gdp'] = ((df.loc[valid_exp_indices, 'total_expenditure'] / df.loc[valid_exp_indices, 'gdp']) * 100).round(2)
df.loc[valid_rev_indices, 'total_revenue_pct_gdp'] = ((df.loc[valid_rev_indices, 'total_revenue'] / df.loc[valid_rev_indices, 'gdp']) * 100).round(2)

## Assign Public Finance Trait
Each Country has a certain public finance trait
https://www.oecd-ilibrary.org/docserver/4d3d8b25-en.pdf?expires=1700814053&id=id&accname=guest&checksum=C108E74662CE152F3313D2CE2EB436FD

In [28]:
# Mapping of ISO 3166-1 alpha-3 codes to group numbers based on the image
iso3_to_group = {
    'DNK': 1, 'FIN': 1, 'NLD': 1, 'SVN': 1, 'SWE': 1,
    'AUT': 2, 'BEL': 2, 'FRA': 2, 'DEU': 2,
    'GRC': 3, 'HUN': 3, 'ITA': 3, 'PRT': 3,
    'ISL': 4, 'IRL': 4, 'ESP': 4, 'GBR': 4,
    'CZE': 5, 'EST': 5, 'POL': 5, 'SVK': 5,
    'LUX': 6, 'NOR': 6, 'CHE': 6,
    'ISR': 7, 'JPN': 7, 'KOR': 7, 'USA': 7
}

df['public_finance_trait'] = df['iso3'].map(iso3_to_group)

### Assign Developing Countries

In [29]:
# List of ISO country codes classified as developing based on common economic indexes
developing_countries = ['BGR', 'CHL', 'COL', 'CRI', 'LTU', 'LVA', 'MEX', 'TUR']

# Create a new column in the DataFrame, default to 0 (developed)
df['developing'] = 0

# Set the value to 1 for developing countries based on their country codes
df.loc[df['iso3'].isin(developing_countries), 'developing'] = 1

# Healthcare type
Countries not listed under any model include:

Australia (AUS)
Bulgaria (BGR)
Chile (CHL)
Colombia (COL)
Costa Rica (CRI)
Greece (GRC)
Hungary (HUN)
Ireland (IRL)
Iceland (ISL)
Israel (ISR)
Lithuania (LTU)
Luxembourg (LUX)
Latvia (LVA)
Mexico (MEX)
Norway (NOR)
Poland (POL)
Portugal (PRT)
Slovakia (SVK)
Slovenia (SVN)
Turkey (TUR)
United States (USA)

In [ ]:
# Mapping of country codes to healthcare model types
# healthcare_model = {
#     'GBR': 1, 'ESP': 1, 'ITA': 1, 'DNK': 1, 'SWE': 1, 'FIN': 1, 'NZL': 1,  # Beveridge Model
#     'DEU': 2, 'AUT': 2, 'BEL': 2, 'CHE': 2, 'CZE': 2, 'NLD': 2, 'FRA': 2, 'JPN': 2, 'KOR': 2,  # Bismarck Model
#     'CAN': 3,  # National Health Insurance Model
#     'USA': 4  # Out-of-Pocket Model
# }

# Assuming 'df' is your DataFrame and it has a column 'country_code' with the ISO country codes
# Default to 4 for out-of-pocket model
# df['healthcare_model_type'] = df['country_code'].map(healthcare_model).fillna(4).astype(int)

# Save CSV

In [30]:
# Save CSV
df.to_csv('oecd_data.csv', index=False)